In [1]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import librosa
import torch

device = "cpu"#torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = Wav2Vec2Processor.from_pretrained("khanhld/wav2vec2-base-vietnamese-160h")
model = Wav2Vec2ForCTC.from_pretrained("khanhld/wav2vec2-base-vietnamese-160h")
model.to(device)

def transcribe(wav):
  input_values = processor(wav, sampling_rate=16000, return_tensors="pt").input_values
  logits = model(input_values.to(device)).logits
  pred_ids = torch.argmax(logits, dim=-1)
  pred_transcript = processor.batch_decode(pred_ids)[0]
  return pred_transcript


C:\Users\minhd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at khanhld/wav2vec2-base-vietnamese-160h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoin

In [ ]:
wav, _ = librosa.load('vi_test.wav', sr = 16000)
print(f"transcript: {transcribe(wav)}")

In [1]:
import sounddevice as sd
from scipy.io.wavfile import write

fs = 16000  # Sample rate
seconds = 5  # Duration of recording

myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=1)
sd.wait()  # Wait until recording is finished
write('output.wav', fs, myrecording)  # Save as WAV file

# wav, _ = librosa.load('output.wav', sr = 16000)
# print(f"transcript: {transcribe(wav)}")
print(myrecording.shape)
print(f"transcript: {transcribe(myrecording[:,0])}")

In [ ]:
from transformers import Wav2Vec2ConformerForCTC, Wav2Vec2Processor
import torch
import librosa

model_id = "facebook/wav2vec2-conformer-rel-pos-large-960h-ft"
device = "cuda"#torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = Wav2Vec2Processor.from_pretrained(model_id)
model = Wav2Vec2ConformerForCTC.from_pretrained(model_id).to(device)
 

def transcribe(wav):
  input_values = processor(wav, sampling_rate=16000, return_tensors="pt").input_values
  logits = model(input_values.to(device)).logits
  pred_ids = torch.argmax(logits, dim=-1)
  pred_transcript = processor.batch_decode(pred_ids)[0]
  return pred_transcript


In [ ]:
wav, _ = librosa.load('en_test.wav', sr = 16000)
print(f"transcript: {transcribe(wav)}")

In [1]:
from transformers import pipeline
import scipy

synthesiser = pipeline("text-to-speech", "suno/bark-small", device='cuda')

C:\Users\minhd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\minhd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [2]:
# speech = synthesiser("The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's attention_mask to obtain reliable results.", forward_params={"do_sample": True})
speech = synthesiser("Hello", forward_params={"do_sample": True})

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [3]:
scipy.io.wavfile.write("bark_out.wav", rate=speech["sampling_rate"], data=speech["audio"][0])

In [9]:
speech

{'audio': array([[-0.00882713, -0.00808206, -0.00861566, ..., -0.00291609,
         -0.0036264 , -0.00253808]], dtype=float32),
 'sampling_rate': 24000}

In [8]:
import sounddevice as sd
import librosa
wav, sr = librosa.load('bark_out.wav', sr = 24000)
sd.play(speech["audio"][0], sr)

In [6]:
print(wav)

[-0.00882713 -0.00808206 -0.00861566 ... -0.00291609 -0.0036264
 -0.00253808]
